In [1]:
import numpy
import keras
from keras_rcnn import datasets, models, preprocessing, utils
from keras_rcnn.datasets import shape
import json
import os
import random
!ls

Using TensorFlow backend.


CheckEnvironment.ipynb	    kernel.ipynb	      Pneumonia_Training.ipynb
config.py		    kernel_LG.ipynb	      __pycache__
create_patient_JSONs.ipynb  logs		      README.md
data			    models		      req.txt
data_preprocessing	    obj_detection_rcnn.ipynb  resources


In [2]:
DATA_PATH = r'data'
JSON_FILE = os.path.join(DATA_PATH,r'train_positive.json')

In [3]:
#load data

with open(JSON_FILE) as j:
    data = json.load(j)
    
data[0]

{'image': {'checksum': '14a05c5cd4498a9beb3bc9ca555c8cd0',
  'pathname': 'data/train_images/positive/4525bf0b-20b4-4f4f-9149-e32632812d89.png',
  'shape': {'r': 1024, 'c': 1024, 'channels': 1}},
 'objects': [{'bounding_box': {'minimum': {'r': 301, 'c': 403},
    'maximum': {'r': 491, 'c': 757}},
   'category': 'sick'},
  {'bounding_box': {'minimum': {'r': 533, 'c': 359},
    'maximum': {'r': 788, 'c': 755}},
   'category': 'sick'}]}

In [4]:
data_shuffled = data[:]

portion = int(0.8*len(data_shuffled))

#shufflee  list
random.seed(55)
random.shuffle(data_shuffled)

training_dictionary = data_shuffled[:20]
test_dictionary = data_shuffled[20:30]

portion

4527

In [5]:
categories = {"sick": 1}

generator = preprocessing.ObjectDetectionGenerator()

generator = generator.flow_from_dictionary(
    dictionary=training_dictionary,
    categories=categories,
    target_size=(224, 224),
    color_mode='rgb'
)

validation_data = preprocessing.ObjectDetectionGenerator()

validation_data = validation_data.flow_from_dictionary(
    dictionary=test_dictionary,
    categories=categories,
    target_size=(224, 224),
    color_mode='rgb'
)


In [6]:
# keras_rcnn.
model = models.RCNN((224, 224, 3), ["pneumonia"])

optimizer = keras.optimizers.Adam()

model.compile(optimizer)

In [7]:
from keras.callbacks import Callback, TerminateOnNaN, TensorBoard, LearningRateScheduler

In [8]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

history = LossHistory()

term_nan = TerminateOnNaN()
    

In [9]:
def UpdateLR(i, lr):
    if (i == 0):
        return 0.001
    elif(i < 10):
        return lr*0.9
    elif(i < 20):
        return lr*0.8

In [10]:
tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
sche = LearningRateScheduler(schedule=UpdateLR, verbose=1)

In [ ]:
model.fit_generator(
    epochs=10,
    generator=generator,
    validation_data=validation_data,
    callbacks=[history,term_nan,tb,sche]
)

/home/claudio/Documents/venv/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10


/home/claudio/Documents/venv/lib/python3.6/site-packages/scikit_image-0.14.0-py3.6-linux-x86_64.egg/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
/home/claudio/Documents/venv/lib/python3.6/site-packages/scikit_image-0.14.0-py3.6-linux-x86_64.egg/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/claudio/Documents/venv/lib/python3.6/site-packages/scikit_image-0.14.0-py3.6-linux-x86_64.egg/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 


Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
20/20 [==============================] - 39s 2s/step - loss: 90.2694 - val_loss: 634578.3451
Epoch 2/10

Epoch 00002: LearningRateScheduler setting learning rate to 0.0009000000427477062.
20/20 [==============================] - 38s 2s/step - loss: 2.3799 - val_loss: 736046.4193
Epoch 3/10

Epoch 00003: LearningRateScheduler setting learning rate to 0.0008100000384729356.
20/20 [==============================] - 38s 2s/step - loss: 1.6441 - val_loss: 206642.0176
Epoch 4/10

Epoch 00004: LearningRateScheduler setting learning rate to 0.0007290000503417104.
12/20 [=================>............] - ETA: 13s - loss: 1.5560

In [ ]:
import skimage

image = skimage.io.imread(r'data/test_images/00a05408-8291-4231-886e-13763e103161.png')



In [ ]:
image.shape